In [6]:
SELECT
	l.name AS league,
	COUNT(m.country_id) AS total_matches
FROM soccer.league AS l
LEFT JOIN soccer.match AS m
USING(country_id)
GROUP BY l.name;

,league,total_matches
0,Switzerland Super League,702
1,Poland Ekstraklasa,960
2,Netherlands Eredivisie,1224
3,Scotland Premier League,912
4,France Ligue 1,1520
5,Spain LIGA BBVA,1520
6,Germany 1. Bundesliga,1224
7,Italy Serie A,1497
8,Portugal Liga ZON Sagres,1026
9,England Premier League,1520


In [7]:
SELECT
	date,
	id,
	home_goal,
	away_goal
FROM soccer.match
WHERE season = '2013/2014';

,date,id,home_goal,away_goal
0,2014-03-29 00:00:00+00:00,1237,2,0
1,2014-03-29 00:00:00+00:00,1238,0,1
2,2014-04-05 00:00:00+00:00,1239,1,0
3,2014-04-05 00:00:00+00:00,1240,0,0
4,2014-04-12 00:00:00+00:00,1241,2,1
...,...,...,...,...
3027,2013-09-26 00:00:00+00:00,25615,1,2
3028,2013-09-25 00:00:00+00:00,25616,0,2
3029,2013-09-25 00:00:00+00:00,25617,2,1
3030,2013-09-25 00:00:00+00:00,25618,0,2


# CASE statements

CASE statements are SQL's version of an "IF this THEN that" statement. 

Case statements have three parts:
- a WHEN clause
- a THEN clause
- an ELSE clause

The WHEN clause -- tests a given condition. If this condition is TRUE, it returns the item you specify after your THEN clause. You can create multiple conditions by listing WHEN and THEN statements within the same CASE statement. The CASE statement is then ended with an ELSE clause that returns a specified value if all of your when statements are not true.

```
CASE WHEN x=1 THEN 'a'
     WHEN x=2 THEN 'b'
     ELSE 'c' END AS new_column 
```

In [8]:
SELECT
	id,
	home_goal,
	away_goal,
	-- Note that all these cases create a new variable
	CASE WHEN home_goal > away_goal THEN 'Home Team Win'
		 WHEN home_goal < away_goal THEN 'Away Team Win'
		 ELSE 'Tie' END AS outcome
FROM soccer.match
WHERE season = '2013/2014';

```
-- Identify the home team as Bayern Munich, Schalke 04, or Other
SELECT 
	CASE WHEN hometeam_id = 10189 THEN 'FC Schalke 04'
             WHEN hometeam_id = 9823 THEN 'FC Bayern Munich'
             ELSE 'Other' END AS home_team,
	COUNT(id) AS total_matches
FROM matches_germany
GROUP BY home_team;

```
SELECT 
	date,
	-- Identify home wins, losses, or ties
	CASE WHEN home_goal > away_goal THEN 'Home win!'
         WHEN home_goal < away_goal THEN 'Home loss :(' 
         ELSE 'Tie' END AS outcome
FROM matches_spain;
```

```
-- Select matches where Barcelona was the away team
SELECT  
	m.date,
	t.team_long_name AS opponent,
	CASE  WHEN m.home_goal < m.away_goal THEN 'Barcelona win!'
        WHEN m.home_goal > m.away_goal THEN 'Barcelona loss :(' 
        ELSE 'Tie' END AS outcome
FROM matches_spain AS m
LEFT JOIN teams_spain AS t 
ON m.hometeam_id = t.team_api_id
-- Filter for Barcelona
WHERE m.awayteam_id = 8634;
```

In [9]:
SELECT date, hometeam_id, awayteam_id,
       CASE WHEN hometeam_id = 8455 AND home_goal > away_goal
				 THEN  'Chelsea home win!'
			WHEN awayteam_id = 8455 AND home_goal < away_goal
				 THEN  'Chelsea away win!'
			ELSE 'Loss or tie' END AS outcome
FROM soccer.match
WHERE hometeam_id = 8455 OR awayteam_id = 8455;

,date,hometeam_id,awayteam_id,outcome
0,2011-08-14 00:00:00+00:00,10194,8455,Loss or tie
1,2011-10-29 00:00:00+00:00,8455,9825,Loss or tie
2,2011-11-05 00:00:00+00:00,8655,8455,Chelsea away win!
3,2011-11-20 00:00:00+00:00,8455,8650,Loss or tie
4,2011-11-26 00:00:00+00:00,8455,8602,Chelsea home win!
...,...,...,...,...
147,2014-09-21 00:00:00+00:00,8456,8455,Loss or tie
148,2014-09-27 00:00:00+00:00,8455,10252,Chelsea home win!
149,2014-10-05 00:00:00+00:00,8455,9825,Chelsea home win!
150,2014-10-18 00:00:00+00:00,9826,8455,Chelsea away win!


When testing logical conditions, it's important to carefully consider which rows of your data are part of your ELSE clause, and if they're categorized correctly. The do so is to ensure adding specific filters in the WHERE clause that exclude objects that do not belong to the conditions that are being evaluated.

# Filtering with CASE

CASE statements allow you to categorize data that you're interested in -- and exclude data you're not interested in. In order to do this, you can use a CASE statement as a filter in the WHERE statement to remove output you don't want to see.

Here is how you might set that up:

```
SELECT *
FROM table
WHERE 
    CASE WHEN a > 5 THEN 'Keep'
         WHEN a <= 5 THEN 'Exclude' END = 'Keep';
```

The following query returns quite a few null results. 

```
SELECT date,
	   CASE WHEN date > '2015-01-01' THEN 'More recent'
			WHEN DATE < '2012-01-01' then 'Older'
			END AS date_category
FROM soccer.match
```

But what if you want to exclude them? In order to filter a query by a CASE statement, you include **the entire CASE statement, except its alias, in WHERE**. You then specify what you want to include, or exclude. 

In [2]:
SELECT date,
	   CASE WHEN date > '2015-01-01' THEN 'More recent'
			WHEN DATE < '2012-01-01' then 'Older'
			END AS date_category
FROM soccer.match
WHERE CASE WHEN date > '2015-01-01' THEN 'More recent'
	  WHEN DATE < '2012-01-01' then 'Older'
      END IS NOT NULL;

,date,date_category
0,2011-07-29 00:00:00+00:00,Older
1,2011-07-30 00:00:00+00:00,Older
2,2011-07-30 00:00:00+00:00,Older
3,2011-07-30 00:00:00+00:00,Older
4,2011-07-30 00:00:00+00:00,Older
...,...,...
3266,2015-05-29 00:00:00+00:00,More recent
3267,2015-05-29 00:00:00+00:00,More recent
3268,2015-05-29 00:00:00+00:00,More recent
3269,2015-05-29 00:00:00+00:00,More recent


```
SELECT 
	date,
	CASE WHEN hometeam_id = 8634 THEN 'FC Barcelona' 
         ELSE 'Real Madrid CF' END as home,
	CASE WHEN awayteam_id = 8634 THEN 'FC Barcelona' 
         ELSE 'Real Madrid CF' END as away,
	-- Identify possible home match outcomes
	CASE WHEN home_goal > away_goal AND hometeam_id = 8634 
	     	THEN 'Barcelona win!'
         WHEN home_goal < away_goal AND awayteam_id = 8633 
		 	THEN 'Real Madrid win!'
         ELSE 'Tie!' END AS outcome
FROM matches_spain
WHERE hometeam_id = 8634 AND awayteam_id = 8633;
```